# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 3 2022 10:27AM,243353,20,8355937,"Exact-Rx, Inc.",Released
1,Aug 3 2022 10:22AM,243352,19,60016466-T1,"GUSA Granules USA, Inc.",Released
2,Aug 3 2022 10:22AM,243352,19,60016466-T2,"GUSA Granules USA, Inc.",Released
3,Aug 3 2022 10:20AM,243351,15,PNC377582A,"Person & Covey, Inc.",Released
4,Aug 3 2022 10:19AM,243350,10,8355420,"Snap Medical Industries, LLC",Released
5,Aug 3 2022 10:18AM,243349,19,ADV80004639,"AdvaGen Pharma, Ltd",Released
6,Aug 3 2022 10:10AM,243348,10,CTF0011348,"Citieffe, Inc.",Released
7,Aug 3 2022 10:10AM,243348,10,CTF0011349,"Citieffe, Inc.",Released
8,Aug 3 2022 10:10AM,243348,10,CTF0011350,"Citieffe, Inc.",Released
9,Aug 3 2022 10:07AM,243347,10,BPS1985.1,Beach Patient Assistance,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,243349,Released,1
40,243350,Released,1
41,243351,Released,1
42,243352,Released,2
43,243353,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243349,NaN,NaN,1.0
243350,NaN,NaN,1.0
243351,NaN,NaN,1.0
243352,NaN,NaN,2.0
243353,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243251,0.0,0.0,24.0
243262,0.0,0.0,1.0
243270,0.0,0.0,1.0
243274,13.0,12.0,2.0
243294,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243251,0,0,24
243262,0,0,1
243270,0,0,1
243274,13,12,2
243294,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243251,0,0,24
1,243262,0,0,1
2,243270,0,0,1
3,243274,13,12,2
4,243294,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243251,,,24
1,243262,,,1
2,243270,,,1
3,243274,13,12,2
4,243294,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 3 2022 10:27AM,243353,20,"Exact-Rx, Inc."
1,Aug 3 2022 10:22AM,243352,19,"GUSA Granules USA, Inc."
3,Aug 3 2022 10:20AM,243351,15,"Person & Covey, Inc."
4,Aug 3 2022 10:19AM,243350,10,"Snap Medical Industries, LLC"
5,Aug 3 2022 10:18AM,243349,19,"AdvaGen Pharma, Ltd"
6,Aug 3 2022 10:10AM,243348,10,"Citieffe, Inc."
9,Aug 3 2022 10:07AM,243347,10,Beach Patient Assistance
10,Aug 3 2022 10:03AM,243313,15,"Virtus Pharmaceuticals, LLC"
12,Aug 3 2022 10:02AM,243307,15,"Carwin Pharmaceutical Associates, LLC"
17,Aug 3 2022 9:55AM,243306,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 3 2022 10:27AM,243353,20,"Exact-Rx, Inc.",,,1
1,Aug 3 2022 10:22AM,243352,19,"GUSA Granules USA, Inc.",,,2
2,Aug 3 2022 10:20AM,243351,15,"Person & Covey, Inc.",,,1
3,Aug 3 2022 10:19AM,243350,10,"Snap Medical Industries, LLC",,,1
4,Aug 3 2022 10:18AM,243349,19,"AdvaGen Pharma, Ltd",,,1
5,Aug 3 2022 10:10AM,243348,10,"Citieffe, Inc.",,,3
6,Aug 3 2022 10:07AM,243347,10,Beach Patient Assistance,,,1
7,Aug 3 2022 10:03AM,243313,15,"Virtus Pharmaceuticals, LLC",,,2
8,Aug 3 2022 10:02AM,243307,15,"Carwin Pharmaceutical Associates, LLC",,,5
9,Aug 3 2022 9:55AM,243306,15,"Mizner Bioscience, LLC",,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 10:27AM,243353,20,"Exact-Rx, Inc.",1,,
1,Aug 3 2022 10:22AM,243352,19,"GUSA Granules USA, Inc.",2,,
2,Aug 3 2022 10:20AM,243351,15,"Person & Covey, Inc.",1,,
3,Aug 3 2022 10:19AM,243350,10,"Snap Medical Industries, LLC",1,,
4,Aug 3 2022 10:18AM,243349,19,"AdvaGen Pharma, Ltd",1,,
5,Aug 3 2022 10:10AM,243348,10,"Citieffe, Inc.",3,,
6,Aug 3 2022 10:07AM,243347,10,Beach Patient Assistance,1,,
7,Aug 3 2022 10:03AM,243313,15,"Virtus Pharmaceuticals, LLC",2,,
8,Aug 3 2022 10:02AM,243307,15,"Carwin Pharmaceutical Associates, LLC",5,,
9,Aug 3 2022 9:55AM,243306,15,"Mizner Bioscience, LLC",7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 10:27AM,243353,20,"Exact-Rx, Inc.",1,,
1,Aug 3 2022 10:22AM,243352,19,"GUSA Granules USA, Inc.",2,,
2,Aug 3 2022 10:20AM,243351,15,"Person & Covey, Inc.",1,,
3,Aug 3 2022 10:19AM,243350,10,"Snap Medical Industries, LLC",1,,
4,Aug 3 2022 10:18AM,243349,19,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 10:27AM,243353,20,"Exact-Rx, Inc.",1.0,NaN,NaN
1,Aug 3 2022 10:22AM,243352,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN
2,Aug 3 2022 10:20AM,243351,15,"Person & Covey, Inc.",1.0,NaN,NaN
3,Aug 3 2022 10:19AM,243350,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Aug 3 2022 10:18AM,243349,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 10:27AM,243353,20,"Exact-Rx, Inc.",1.0,0.0,0.0
1,Aug 3 2022 10:22AM,243352,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0
2,Aug 3 2022 10:20AM,243351,15,"Person & Covey, Inc.",1.0,0.0,0.0
3,Aug 3 2022 10:19AM,243350,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Aug 3 2022 10:18AM,243349,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4379664,146.0,12.0,13.0
15,1459833,52.0,0.0,0.0
16,243346,0.0,1.0,0.0
18,243325,1.0,0.0,0.0
19,1216675,19.0,13.0,2.0
20,243353,1.0,0.0,0.0
21,1459913,5.0,1.0,0.0
22,486688,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4379664,146.0,12.0,13.0
1,15,1459833,52.0,0.0,0.0
2,16,243346,0.0,1.0,0.0
3,18,243325,1.0,0.0,0.0
4,19,1216675,19.0,13.0,2.0
5,20,243353,1.0,0.0,0.0
6,21,1459913,5.0,1.0,0.0
7,22,486688,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,146.0,12.0,13.0
1,15,52.0,0.0,0.0
2,16,0.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,19.0,13.0,2.0
5,20,1.0,0.0,0.0
6,21,5.0,1.0,0.0
7,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,146.0
1,15,Released,52.0
2,16,Released,0.0
3,18,Released,1.0
4,19,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,13.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Executing,12.0,0.0,1.0,0.0,13.0,0.0,1.0,1.0
Released,146.0,52.0,0.0,1.0,19.0,1.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,13.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,12.0,0.0,1.0,0.0,13.0,0.0,1.0,1.0
2,Released,146.0,52.0,0.0,1.0,19.0,1.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,13.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,12.0,0.0,1.0,0.0,13.0,0.0,1.0,1.0
2,Released,146.0,52.0,0.0,1.0,19.0,1.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()